In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

Using TensorFlow backend.


In [2]:
!nvidia-smi

Thu Jun 20 07:15:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   48C    P8     9W / 200W |    185MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
print(tf.test.is_gpu_available())

True


In [4]:
def sanitize(string):
    words = string.split(' ')
    return words

In [5]:
#this is the test set
testTb = Table()
list1 = []
list2 = []

with open("test.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        testTb=testTb.with_columns(list1[0][ele], list2[ele])


testX = testTb.select(1, 2)
testY = testTb.select(3)

In [6]:
#this is the training set
tb = Table()
list1 = []
list2 = []

with open("train.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        tb=tb.with_columns(list1[0][ele], list2[ele])
        


trainX = tb.select(1, 2)
trainY = tb.select(3)



In [7]:
#read in the csv files 


train = pd.read_csv("train.tsv", sep='\t')
dev = pd.read_csv("dev.tsv", sep='\t')
test = pd.read_csv("test.tsv", sep='\t')

print(train[:5])

train['title1_tokenized'] = \
    train.loc[:, 'sentence1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sentence2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sentence1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sentence2'] \
         .apply(sanitize)


test['title1_tokenized'] = \
    test.loc[:, 'sentence1'] \
         .apply(sanitize)
test['title2_tokenized'] = \
    test.loc[:, 'sentence2'] \
         .apply(sanitize)

#train_corpus_x1=tb.select(1)
#train_corpus_x2=tb.select(2)
#test_corpus_x1=testTb.select(1)
#test_corpus_x2=testTb.select(2)
#corpus = pd.concat([train_corpus_x1, train_corpus_x2, test_corpus_x1, test_corpus_x2])

   id                                          sentence1  \
0   1  In Paris , in October 1560 , he secretly met t...   
1   2  The NBA season of 1975 -- 76 was the 30th seas...   
2   3  There are also specific discussions , public p...   
3   4  When comparable rates of flow can be maintaine...   
4   5  It is the seat of Zerendi District in Akmola R...   

                                           sentence2  label  
0  In October 1560 , he secretly met with the Eng...      0  
1  The 1975 -- 76 season of the National Basketba...      1  
2  There are also public discussions , profile sp...      0  
3  The results are high when comparable flow rate...      1  
4  It is the seat of the district of Zerendi in A...      1  


In [8]:
#create a dictionary

MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

train_corpus_x1 = train.title1_tokenized
train_corpus_x2 = train.title2_tokenized
train_corpus = pd.concat([
    train_corpus_x1, train_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)
x1_train = tokenizer \
    .texts_to_sequences(train_corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(train_corpus_x2)


print(tokenizer.document_count)




dev_corpus_x1 = dev.title1_tokenized
dev_corpus_x2 = dev.title2_tokenized
dev_corpus = pd.concat([
    dev_corpus_x1, dev_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x2)


test_corpus_x1 = test.title1_tokenized
test_corpus_x2 = test.title2_tokenized
test_corpus = pd.concat([
    test_corpus_x1, test_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(test_corpus)
tokenizer.texts_to_sequences(test_corpus)
x1_test = tokenizer \
    .texts_to_sequences(test_corpus_x1)
x2_test = tokenizer \
    .texts_to_sequences(test_corpus_x2)

word_index = tokenizer.word_index


print(tokenizer.document_count)



98802
130802


In [9]:
#padding and one-hot

max_seq_len1 = max([
    len(seq) for seq in x1_train])
print(max_seq_len1)   #36

MAX_SEQUENCE_LENGTH = 36  #better to have words covered than uncovered
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)


y_train = train.label

y_train = keras \
    .utils \
    .to_categorical(y_train, num_classes=2)


y_dev = dev.label

y_dev = keras \
    .utils \
    .to_categorical(y_dev, num_classes=2)



y_test = test.label

y_test = keras \
    .utils \
    .to_categorical(y_test, num_classes=2)

x1_test[:3]

36


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   43,
           8,    7,   97,    6, 6066, 5611, 3080,    2,  446,   29, 6910,
           4,    5, 3542,  683,   49, 3108, 1221,    4, 4527, 2922, 3890,
          10,    1,    3],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,   22,  131,
        2253,   10, 1628,    4, 2550,   47,  275,   61,   22,  224,  108,
        3874,    5,  141,    1, 1968,    6,    1,   67,  683,   34, 1742,
          10,  280,    3],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    4,  120,  151,    2,    1, 1180, 1172,  198,
           6, 5608,  905,    2,    2,    1, 3980,  304,   15,   32,  269,
        2334, 2052,    3]], dtype=int32)

In [10]:
print("Training Set")
print("-" * 10)
print(np.shape(x1_train))
print(np.shape(x2_train))
print(np.shape(y_train))
print(np.shape(train_corpus))

print("-" * 10)
print(np.shape(x1_test))
print(np.shape(x2_test))
print(np.shape(y_test))
print(np.shape(test_corpus))

print("Test Set")


Training Set
----------
(49401, 36)
(49401, 36)
(49401, 2)
(98802,)
----------
(8000, 36)
(8000, 36)
(8000, 2)
(16000,)
Test Set


In [38]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



Found 400000 word vectors.


In [39]:
NUM_EMBEDDING_DIM = 100
print(len(word_index)+1)
embedding_matrix = np.zeros((len(word_index)+1, NUM_EMBEDDING_DIM))


for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

34723


In [ ]:
#Siamese with GLoVE
NUM_CLASSES=2 #boolean; 1 or 0

 #rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(len(word_index)+1,
                            NUM_EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 5

history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=([x1_test, x2_test], y_test),
    
    shuffle=True
)

In [ ]:
#Naive Siamese
NUM_CLASSES=2 #boolean; 1 or 0

 #rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM, mask_zero=True)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 5

history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=([x1_test, x2_test], y_test),
    
    shuffle=True
)

In [11]:
#BiLSTM?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 20

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_Wiki', save_best_only=True, monitor='val_loss', mode='min')





Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 36, 256)      2560000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None

In [18]:
!nvidia-smi


Thu Jun 20 07:14:21 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   46C    P8    13W / 200W |    582MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
        
history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping, mcp_save], validation_data=([x1_dev, x2_dev], y_dev),
    
    shuffle=True
)

Instructions for updating:
Use tf.cast instead.
Train on 49401 samples, validate on 8000 samples
Epoch 1/20
49401/49401 [==============================] - 24s 490us/step - loss: 0.6810 - acc: 0.5756 - val_loss: 0.6828 - val_acc: 0.5647
Epoch 2/20
49401/49401 [==============================] - 22s 443us/step - loss: 0.6465 - acc: 0.6136 - val_loss: 0.6885 - val_acc: 0.5613
Epoch 3/20
49401/49401 [==============================] - 22s 442us/step - loss: 0.6301 - acc: 0.6273 - val_loss: 0.6907 - val_acc: 0.5649
Epoch 4/20
49401/49401 [==============================] - 22s 442us/step - loss: 0.6196 - acc: 0.6311 - val_loss: 0.6993 - val_acc: 0.5541
Epoch 5/20
49401/49401 [==============================] - 22s 442us/step - loss: 0.5915 - acc: 0.6041 - val_loss: 0.7082 - val_acc: 0.5154
Epoch 6/20
49401/49401 [==============================] - 22s 442us/step - loss: 0.5356 - acc: 0.6087 - val_loss: 0.7020 - val_acc: 0.5073
Epoch 7/20
49401/49401 [==============================] - 22s 442us/s

In [13]:
predicts = model.predict(
    [x1_test, x2_test])




In [14]:


print(len(predicts))

8000


In [15]:
real1=test.sentence1
real2=test.sentence2
realCat=test.label
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(8000):
    if (np.argmax(predicts[x])!=(test.label[x])):
        no=np.append(no, x)
        err1=np.append(err1, real1[x])
        err2=np.append(err2, real2[x])
        wrongCat = np.append(wrongCat, realCat[x])
print(len(wrongCat))

3536


In [35]:
error_analysis = Table()


error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "classification", wrongCat)

id,sent1,sent2,classification
0,"This was a series of nested angular standards , so that ...","This was a series of nested polar scales , so that measu ...",0
1,His father emigrated to Missouri in 1868 but returned wh ...,"His father emigrated to America in 1868 , but returned w ...",0
2,"In January 2011 , the Deputy Secretary General of FIBA A ...","In January 2011 , FIBA Asia deputy secretary general Hag ...",1
3,"Steiner argued that , in the right circumstances , the s ...",Steiner held that the spiritual world can be researched ...,0
4,"Luciano Williames Dias ( born July 25 , 1970 ) is a Braz ...",Luciano Williames Dias ( born 25 July 1970 ) is a former ...,0
6,The smallest number that can be represented in two posit ...,The smallest number that can be represented as a sum of ...,0
8,The Villa Pesquera facilities are owned by the Municipal ...,The facilities of Villa Pesquera are operated by the Mun ...,0
11,He appeared as General Tao in the Amazon - Show `` The M ...,He appeared as General Onoda in the Amazon - Show `` The ...,0
14,"To get there , take the Marine Drive from the Lions Gate ...","To get there , take Marine Drive west from the Lions Gat ...",0
17,"For ideal gases , the molar volume is given by the ideal ...",For ideal gases the molar volume is given by the standar ...,0


In [1]:
clearer = error_analysis.take("sent1", "sent2")

NameError: name 'error_analysis' is not defined

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_with_glove.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_with_glove.h5")        ##first attempt
print("Saved model to disk")           



In [ ]:
#embedding layer output dimension is arbitrary?




In [ ]:
sanitize("I like, hue")

In [5]:
!nvidia-smi

Tue Jun 18 06:50:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   46C    P8     8W / 200W |    719MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [74]:
#possible optimization strats:

#tokenization without punctuation aka modify the dictionary
#tweak MAX_SEQUENCE_LENGTH 
#masking
#NUM_EMBEDDING_DIMENSION
#"optimizer" in the compile function
#loss function
#a different activation function?
#sth other than word level tokenization?
#train on other databases?
#pretrained word embedding
#check errors on the test set?
#model attention seq2seq
#GLOVE dimension modification
#more sanitization?

In [ ]:
# model adaptation: https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92-3-%E6%AD%A5%E9%A9%9F

In [ ]:
#https://paperswithcode.com/paper/multi-task-deep-neural-networks-for-natural


In [ ]:
#attempts: with self trained word embedder, glove embedder, zero masking, 

In [79]:
# We randomly select 5,000 paraphrases and 5,000 non-paraphrases as the dev set, and sample
#another 5,000 paraphrases and 5,000 non-paraphrases as the
#test set. We keep the remaining instances as the training set 2